<a href="https://colab.research.google.com/github/leokao960811/ProgrammingLang/blob/main/W3_APIChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://ai.google.dev/

https://ai.google.dev/gemini-api/docs/quickstart?lang=python

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [4]:
from google.colab import userdata
apikey = userdata.get('test')

In [22]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-2.0-flash")

response = model.generate_content("Write a short story in 50 words.")
print(response.text)

The old lighthouse keeper, Silas, found a message in a bottle washed ashore. "SOS. Abandon ship. Lost at sea." A shiver ran down his spine. He'd heard no distress call. Grabbing his binoculars, he scanned the horizon, a dark storm gathering on the edge of the world. He was not alone.



Sheets 載入

In [12]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1au_UL4MpOzO9ln_6cuIC6Oa_Kozuel3nBtIadjy5Xr4/edit?usp=sharing')

In [13]:
gsheets

<Spreadsheet 'W2' id:1au_UL4MpOzO9ln_6cuIC6Oa_Kozuel3nBtIadjy5Xr4>

In [14]:
from gspread_dataframe import set_with_dataframe

# 選擇工作表
worksheet = gsheets.get_worksheet(1)  # 其他工作表請用 get_worksheet(index)

data = worksheet.get_all_values()
df = pd.DataFrame(data)

df

,0,1,2,3
0,座號,數學,英文,國文
1,1,78,74,63
2,2,81,63,52
3,3,63,59,89
4,4,98,100,90
5,5,77,97,62
6,6,61,80,63
7,7,86,67,62
8,8,93,85,89
9,9,87,91,71


Gemini分析

In [15]:
processdata = df.to_string()

'     0   1    2   3\n0   座號  數學   英文  國文\n1    1  78   74  63\n2    2  81   63  52\n3    3  63   59  89\n4    4  98  100  90\n5    5  77   97  62\n6    6  61   80  63\n7    7  86   67  62\n8    8  93   85  89\n9    9  87   91  71\n10  10  72   65  66'

In [20]:
prompt = f'以下內容為10位學生的期中考成績，請幫我將進行分析、摘要: {processdata} ，同時請預測資料中學生的學習趨勢，並用繁體中文輸出'

In [25]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-2.0-flash")

response = model.generate_content(prompt)
print(response.text)

好的，以下是針對這 10 位學生期中考成績的分析、摘要以及學習趨勢預測：

**數據摘要與分析：**

| 科目 | 平均分數 | 最高分 | 最低分 | 標準差 |
|---|---|---|---|---|
| 數學 | 79.6 | 98 | 61 | 11.54 |
| 英文 | 78.1 | 100 | 59 | 14.04 |
| 國文 | 70.7 | 90 | 52 | 12.37 |

*   **整體表現：** 整體來看，數學和英文的平均分數都接近 80 分，表現相對較好。國文平均分數略低，為 70.7 分。

*   **個別科目分析：**
    *   **數學：** 最高分 98 分，最低分 61 分，差距較大，表示學生間的數學能力差異較明顯。標準差 11.54 顯示分數分布相對分散。
    *   **英文：** 最高分 100 分，最低分 59 分，差距也較大。標準差 14.04 高於數學，代表學生英文程度的差異更大。
    *   **國文：** 最高分 90 分，最低分 52 分，差距也較大。標準差 12.37 在三科中居中，表示國文分數的集中程度介於數學和英文之間。

*   **最高分表現：** 有兩位學生的單科成績達到滿分或接近滿分 (英文 100 分，數學 98 分)，顯示學生在特定科目具有優異潛力。

*   **最低分表現：** 各科都有學生低於 60 分，需要特別關注這些學生，了解他們在學習上遇到的困難並提供協助。

**學生學習趨勢預測：**

基於現有的數據，我們可以初步預測以下學習趨勢：

1.  **個別科目能力差異明顯：** 學生在不同科目之間的表現差異較大。有些學生可能在數學或英文方面表現突出，但在國文方面較弱，反之亦然。因此，需要針對學生的個別強弱項進行差異化教學。

2.  **兩極分化可能性：** 由於各科的標準差較大，如果沒有適當的輔導和協助，學生間的成績差距可能會持續擴大，形成兩極分化的現象。

3.  **學習策略需求：** 由於國文平均分數相對較低，可能需要加強學生的閱讀理解、寫作技巧和語文基礎知識。同時，也要鼓勵學生發展更有效的學習策略，例如時間管理、筆記技巧、複習方法等。

4.  **潛力發展空間：** 有部分學生在特定科目表現優異，可以進一步挖掘他們的潛力，鼓勵他們參加相關的競賽

In [30]:
from io import StringIO

df2=pd.read_csv(StringIO(response.text))

worksheet2 = gsheets.get_worksheet(3)
set_with_dataframe(worksheet2, df2)
df2

,好的，以下是針對這 10 位學生期中考成績的分析、摘要以及學習趨勢預測：
0,**數據摘要與分析：**
1,| 科目 | 平均分數 | 最高分 | 最低分 | 標準差 |
2,|---|---|---|---|---|
3,| 數學 | 79.6 | 98 | 61 | 11.54 |
4,| 英文 | 78.1 | 100 | 59 | 14.04 |
5,| 國文 | 70.7 | 90 | 52 | 12.37 |
6,* **整體表現：** 整體來看，數學和英文的平均分數都接近 80 分，表現相對較好。國...
7,* **個別科目分析：**
8,* **數學：** 最高分 98 分，最低分 61 分，差距較大，表示學生間的數...
9,* **英文：** 最高分 100 分，最低分 59 分，差距也較大。標準差 1...
